In [ ]:
import pygame
import random
from typing import List

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 800, 600
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
BAR_WIDTH = 5

class Visualizer:
    def __init__(self):
        self.window = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption("Sorting Visualizer")
        self.clock = pygame.time.Clock()
        self.array = self.generate_array()
        self.sorting = False
        self.current_algo = "bubble"

    def generate_array(self) -> List[int]:
        return [random.randint(50, HEIGHT-50) for _ in range(WIDTH // BAR_WIDTH)]

    def draw_bars(self, highlight: set = None) -> None:
        self.window.fill(WHITE)
        for i, val in enumerate(self.array):
            color = BLUE
            if highlight and i in highlight:
                color = RED
            pygame.draw.rect(self.window, color, 
                           (i * BAR_WIDTH, HEIGHT - val, BAR_WIDTH, val))
        pygame.display.update()

    def bubble_sort(self) -> None:
        n = len(self.array)
        for i in range(n):
            for j in range(0, n-i-1):
                if self.array[j] > self.array[j+1]:
                    self.array[j], self.array[j+1] = self.array[j+1], self.array[j]
                    self.draw_bars({j, j+1})
                    self.clock.tick(60)  # Control visualization speed
                    yield True  # Generator to maintain sort state

    def quick_sort_partition(self, low: int, high: int) -> int:
        i = low - 1
        pivot = self.array[high]

        for j in range(low, high):
            if self.array[j] <= pivot:
                i += 1
                self.array[i], self.array[j] = self.array[j], self.array[i]
                self.draw_bars({i, j, high})
                self.clock.tick(60)
        self.array[i+1], self.array[high] = self.array[high], self.array[i+1]
        return i + 1

    def quick_sort(self, low: int = 0, high: int = None) -> None:
        if high is None:
            high = len(self.array) - 1
        if low < high:
            pi = self.quick_sort_partition(low, high)
            yield from self.quick_sort(low, pi-1)
            yield from self.quick_sort(pi+1, high)
        yield False

    def run(self) -> None:
        generator = None
        running = True
        
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_r:  # Reset array
                        self.array = self.generate_array()
                        self.sorting = False
                    elif event.key == pygame.K_b:  # Bubble sort
                        self.current_algo = "bubble"
                        generator = self.bubble_sort()
                        self.sorting = True
                    elif event.key == pygame.K_q:  # Quick sort
                        self.current_algo = "quick"
                        generator = self.quick_sort()
                        self.sorting = True
                    elif event.key == pygame.K_ESCAPE:
                        running = False

            if self.sorting and generator:
                try:
                    next(generator)
                except StopIteration:
                    self.sorting = False
                    self.draw_bars()
                    pygame.display.update()
            else:
                self.draw_bars()

            pygame.display.flip()

        pygame.quit()

if __name__ == "__main__":
    vis = Visualizer()
    vis.run()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
import pygame
import random
import time
from pygame.locals import *
from typing import List, Generator, Tuple

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 1200, 800
COLORS = {
    'background': (28, 28, 28),
    'bar': (94, 129, 162),
    'highlight1': (255, 107, 107),
    'highlight2': (155, 246, 255),
    'sorted': (181, 234, 215),
    'text': (255, 255, 255)
}
BAR_WIDTH = 4
BUTTONS = {
    'bubble': (50, 50, 120, 40),
    'selection': (180, 50, 120, 40),
    'merge': (310, 50, 120, 40),
    'quick': (440, 50, 120, 40),  
    'reset': (440, 50, 120, 40)
}

class SortingVisualizer:
    def __init__(self):
        self.window = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption("Sorting Visualizer")
        self.clock = pygame.time.Clock()
        self.array = self.generate_array()
        self.sorting = False
        self.current_algo = None
        self.stats = {'comparisons': 0, 'swaps': 0, 'time': 0}
        self.speed = 60  # FPS

    def generate_array(self) -> List[int]:
        return [random.randint(50, HEIGHT-100) for _ in range(WIDTH // BAR_WIDTH)]

    def draw_interface(self) -> None:
        # Draw buttons
        for name, rect in BUTTONS.items():
            color = (100, 200, 100) if name == self.current_algo else (70, 70, 70)
            pygame.draw.rect(self.window, color, rect)
            font = pygame.font.Font(None, 24)
            text = font.render(name.capitalize(), True, COLORS['text'])
            self.window.blit(text, (rect[0]+10, rect[1]+10))

        # Draw stats
        font = pygame.font.Font(None, 28)
        stats_text = [
            f"Comparisons: {self.stats['comparisons']}",
            f"Swaps: {self.stats['swaps']}",
            f"Time: {self.stats['time']:.1f}s",
            f"Speed: {self.speed}FPS"
        ]
        for i, text in enumerate(stats_text):
            surf = font.render(text, True, COLORS['text'])
            self.window.blit(surf, (WIDTH-250, 50 + i*30))

    def draw_bars(self, highlights: Tuple[set, set] = None) -> None:
        self.window.fill(COLORS['background'])
        
        for i, val in enumerate(self.array):
            color = COLORS['bar']
            if highlights:
                if i in highlights[0]:
                    color = COLORS['highlight1']
                elif i in highlights[1]:
                    color = COLORS['highlight2']
            pygame.draw.rect(self.window, color, 
                           (i * BAR_WIDTH, HEIGHT - val, BAR_WIDTH-1, val))
        
        self.draw_interface()
        pygame.display.update()

    def bubble_sort(self) -> Generator:
        self.stats = {'comparisons': 0, 'swaps': 0, 'time': time.time()}
        n = len(self.array)
        for i in range(n):
            for j in range(0, n-i-1):
                self.stats['comparisons'] += 1
                if self.array[j] > self.array[j+1]:
                    self.array[j], self.array[j+1] = self.array[j+1], self.array[j]
                    self.stats['swaps'] += 1
                    yield ({j, j+1}, set())
                self.clock.tick(self.speed)
        self.stats['time'] = time.time() - self.stats['time']

    def selection_sort(self) -> Generator:
        self.stats = {'comparisons': 0, 'swaps': 0, 'time': time.time()}
        n = len(self.array)
        for i in range(n):
            min_idx = i
            for j in range(i+1, n):
                self.stats['comparisons'] += 1
                if self.array[j] < self.array[min_idx]:
                    min_idx = j
                yield ({i, j}, set())
                self.clock.tick(self.speed)
            self.array[i], self.array[min_idx] = self.array[min_idx], self.array[i]
            self.stats['swaps'] += 1
            yield ({i, min_idx}, set())
        self.stats['time'] = time.time() - self.stats['time']

    def merge_sort_gen(self, arr=None, left=0) -> Generator:
        if arr is None:
            self.stats = {'comparisons': 0, 'swaps': 0, 'time': time.time()}
            arr = self.array.copy()
        
        if len(arr) > 1:
            mid = len(arr) // 2
            left_arr = arr[:mid]
            right_arr = arr[mid:]

            yield from self.merge_sort_gen(left_arr, left)
            yield from self.merge_sort_gen(right_arr, left + mid)

            i = j = k = 0
            while i < len(left_arr) and j < len(right_arr):
                self.stats['comparisons'] += 1
                if left_arr[i] < right_arr[j]:
                    arr[k] = left_arr[i]
                    i += 1
                else:
                    arr[k] = right_arr[j]
                    j += 1
                k += 1
                self.array[left:left+len(arr)] = arr
                self.stats['swaps'] += 1
                yield ({left + k}, set())
                self.clock.tick(self.speed)

            while i < len(left_arr):
                arr[k] = left_arr[i]
                i += 1
                k += 1
                self.array[left:left+len(arr)] = arr
                yield ({left + k}, set())
                self.clock.tick(self.speed)

            while j < len(right_arr):
                arr[k] = right_arr[j]
                j += 1
                k += 1
                self.array[left:left+len(arr)] = arr
                yield ({left + k}, set())
                self.clock.tick(self.speed)
        
        if left == 0:
            self.stats['time'] = time.time() - self.stats['time']

    def quick_sort(self, low=0, high=None) -> Generator:
    if high is None:
        self.stats = {'comparisons': 0, 'swaps': 0, 'time': time.time()}
        high = len(self.array) - 1
    
    if low < high:
        pi = yield from self.partition(low, high)
        yield from self.quick_sort(low, pi-1)
        yield from self.quick_sort(pi+1, high)
    
    if low == 0 and high == len(self.array)-1:
        self.stats['time'] = time.time() - self.stats['time']

def partition(self, low: int, high: int) -> Generator:
    pivot = self.array[high]
    i = low - 1
    
    for j in range(low, high):
        self.stats['comparisons'] += 1
        if self.array[j] <= pivot:
            i += 1
            self.array[i], self.array[j] = self.array[j], self.array[i]
            self.stats['swaps'] += 1
            yield ({i, j, high}, set())  # Highlight pivot + swapped elements
            self.clock.tick(self.speed)
    
    self.array[i+1], self.array[high] = self.array[high], self.array[i+1]
    self.stats['swaps'] += 1
    yield ({i+1, high}, set())  # Final pivot position
    return i + 1

    def handle_events(self) -> bool:
        for event in pygame.event.get():
            if event.type == QUIT:
                return False
            if event.type == MOUSEBUTTONDOWN:
                x, y = pygame.mouse.get_pos()
                for name, rect in BUTTONS.items():
                    if rect[0] <= x <= rect[0]+rect[2] and rect[1] <= y <= rect[1]+rect[3]:
                        self.handle_button(name)
            if event.type == KEYDOWN:
                if event.key == K_UP:
                    self.speed = min(240, self.speed + 10)
                elif event.key == K_DOWN:
                    self.speed = max(10, self.speed - 10)
        return True

    def handle_button(self, name: str) -> None:
        if name == 'reset':
            self.array = self.generate_array()
            self.sorting = False
            self.current_algo = None
        elif not self.sorting:
            self.current_algo = name
            self.sorting = True
            self.generator = {
                'bubble': self.bubble_sort,
                'merge': self.merge_sort_gen,
                'selection': self.selection_sort,
                'quick': self.quick_sort  # Add this line

            }[name]()

    def run(self) -> None:
        running = True
        while running:
            running = self.handle_events()

            if self.sorting:
                try:
                    highlights = next(self.generator)
                    self.draw_bars(highlights)
                except StopIteration:
                    self.sorting = False
                    self.draw_bars((set(), set()))
            else:
                self.draw_bars()

            pygame.display.flip()
            self.clock.tick(60)

        pygame.quit()

if __name__ == "__main__":
    vis = SortingVisualizer()
    vis.run()

IndentationError: expected an indented block after function definition on line 156 (3868417901.py, line 157)